In [4]:
import cv2
import mediapipe as mp
from math import sqrt

In [14]:
medhands=mp.solutions.hands
mpdraw=mp.solutions.drawing_utils

hand=medhands.Hands(max_num_hands=1)

video=cv2.VideoCapture(0)
while True:
    suc,img=video.read()
    img1=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    res=hand.process(img1)

    tip_ids=[4,8,12,16,20]
    lmlist=[]
    gesture=""

    if res.multi_hand_landmarks:
        for handlms in res.multi_hand_landmarks:

            for id,lm in enumerate(handlms.landmark):
                cx=lm.x
                cy=lm.y
                lmlist.append([id,cx,cy])

            if len(lmlist)!=0 and len(lmlist)==21: #all landmarks are detected
                    fingercount=[]

                    if lmlist[12][1]<lmlist[20][1]: #right hand
                        if lmlist[4][1]>lmlist[3][1]: #thumb closed
                            fingercount.append(0)
                        else: #thumb open
                            fingercount.append(1) 
                    else: #left hand
                        if lmlist[4][1]<lmlist[3][1]: #closed
                            fingercount.append(0)
                        else: #open
                            fingercount.append(1)

                    for i in range(1,5): #fingers except thumb
                        if lmlist[tip_ids[i]][2] > lmlist[tip_ids[i]-2][2] : #finger closed
                            fingercount.append(0)
                        else: #finger open
                            fingercount.append(1)

                    # print(fingercount)

                    #Gestures

                    thumb_y=lmlist[4][2]
                    wrist_y = lmlist[0][2]
                    index_tip = lmlist[8]
                    thumb_tip = lmlist[4]

                    if fingercount == [1, 0, 0, 0, 0]:
                        if thumb_y < wrist_y:
                            gesture = "Thumbs Up"
                        else:
                            gesture = "Thumbs Down"
                    elif fingercount == [0, 1, 1, 0, 0]:
                        gesture = "Peace"
                    elif fingercount == [1, 1, 1, 1, 1]:
                        gesture = "Stop"
                    elif fingercount == [1, 1, 0, 0, 1]:
                        gesture = "ILY"
                    else:
                        x1, y1 = thumb_tip[1], thumb_tip[2]
                        x2, y2 = index_tip[1], index_tip[2]
                        dist = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
                        if dist < 40:  # Pixel distance threshold
                            gesture = "OK"

                    if gesture:
                        cv2.putText(img, gesture, (35, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 6)
                    

            mpdraw.draw_landmarks(img,handlms,medhands.HAND_CONNECTIONS)

    cv2.imshow('Gestures',img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()